# All convoliutional network

Šios architketūros implementacijoje yra daugmaž viskas labai panašu į VGG16 išskyrus ta dalis kuriome aprašome allconvnet architektūra.

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

In [ ]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
sys.path.append('/content/drive/My Drive/Colab Notebooks')

In [ ]:
from helper_evaluation import set_all_seeds, set_deterministic, compute_confusion_matrix
from helper_train_extended import train_model
from helper_plotting_extended import plot_training_loss, plot_accuracy, show_examples, plot_confusion_matrix
from helper_dataset_extended import get_dataloaders_cifar10, UnNormalize

In [ ]:
RANDOM_SEED = 123
BATCH_SIZE  = 256
NUM_EPOCHS  = 50
DEVICE = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')

In [ ]:
train_transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((70, 70)),
    torchvision.transforms.RandomCrop((64, 64)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                      ])

test_transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((70, 70)),        
    torchvision.transforms.CenterCrop((64, 64)),            
    torchvision.transforms.ToTensor(),                
    torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


train_loader, valid_loader, test_loader = get_dataloaders_cifar10(
    batch_size=BATCH_SIZE,
    validation_fraction=0.1,
    train_transforms=train_transforms,
    test_transforms=test_transforms,
    num_workers=2)

for images, labels in train_loader:  
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    print('Class labels of 10 examples:', labels[:10])
    break

Šioje architektūroje turime tik konvoliucinius sluoksnius su BatchNorm normalizacijos operacijomis ir ReLU aktivacijomis. Nėra  MaxPool2d operacijų ir nėra MLP(multi layer perceptron) FC(fully connected) sluoksnio. 

 - Pirmoji konvoliucija padidina kanalų skaičių (nes yra 16 filtrų) bet išlaiko įvesties ir išvesties matmenys vienodus dėka užpildo `padding`. Įvestis yra 70x70, išvestis taip pat bus 70x70. 
 - Sekanti konvoliucija kurios kanalų įvestis ir išvestis yra identiška, bet kadangi šios konvoliucijos žignis `stride` yra 2 tai išvestis sumažės dvigubai, tai `maxpooling` atitkmuo.
 - Trečioji konvoliucija vėlgi padidina kanalų skaičių, ir išlaiko įvesties ir išvesties matmenys identiškus
 - Ketvirtoji konvoliucija tik redukuoja įšvestį sumažindama matmenys dvigubai
 
Ir tai kartojasi keletą kartų toliau. Ir galutianme etape turime 10-ies kategorijų etiketes nes naudojame CIFAR-10 todėl paskutinio konvoliucinio sluoksnio išvestis yra kategorijų etikečiu skaičius, t.y 10.<br>
Pytorch neturi `Global average pooling` kas mums suskaičiuoti kanalų vidurkį, tą galėtume techniški patys suprogramuoti bet tai pat gali tiesiog pritaikyti vidurkinimo sluoksnį su gana universalaus `AdaptiveAvgPool2d` metodo pagalba su įvestimis 1, tai turės principe panašu efektą. 

In [ ]:
class AllConvNet(torch.nn.Module):

    def __init__(self, num_classes):
        super().__init__()
        
        self.net = torch.nn.Sequential(
                      torch.nn.Conv2d(in_channels=3,
                                      out_channels=16,
                                      kernel_size=3,
                                      stride=1,
                                      padding=1,
                                      bias=False),
                      torch.nn.BatchNorm2d(16),
                      torch.nn.ReLU(inplace=True),
                      
                      # vietoje maxpooling
                      torch.nn.Conv2d(in_channels=16,
                                      out_channels=16,
                                      kernel_size=3,
                                      stride=2,
                                      padding=1,
                                      bias=False),   
                      torch.nn.BatchNorm2d(16),
                      torch.nn.ReLU(inplace=True),
            
                      torch.nn.Conv2d(in_channels=16,
                                      out_channels=32,
                                      kernel_size=3,
                                      stride=1,
                                      padding=1,
                                      bias=False),        
                      torch.nn.BatchNorm2d(32),
                      torch.nn.ReLU(inplace=True), 
            
                      # vietoje maxpooling
                      torch.nn.Conv2d(in_channels=32,
                                      out_channels=32,
                                      kernel_size=3,
                                      stride=2,
                                      padding=1,
                                      bias=False),      
                      torch.nn.BatchNorm2d(32),
                      torch.nn.ReLU(inplace=True), 
            
                      torch.nn.Conv2d(in_channels=32,
                                      out_channels=64,
                                      kernel_size=3,
                                      stride=1,
                                      padding=1,
                                      bias=False),   
                      torch.nn.BatchNorm2d(64),
                      torch.nn.ReLU(inplace=True),
            
                      # vietoje maxpooling
                      torch.nn.Conv2d(in_channels=64,
                                      out_channels=64,
                                      kernel_size=3,
                                      stride=2,
                                      padding=1,
                                      bias=False),     
                      torch.nn.BatchNorm2d(64),
                      torch.nn.ReLU(inplace=True),
            
                      torch.nn.Conv2d(in_channels=64,
                                      out_channels=num_classes,
                                      kernel_size=3,
                                      stride=1,
                                      padding=1,
                                      bias=False),    
                      torch.nn.BatchNorm2d(10),
                      torch.nn.ReLU(inplace=True),
            
                      torch.nn.AdaptiveAvgPool2d(1),
                      torch.nn.Flatten()
    )


    def forward(self, x):
        
        x = self.net(x)
        #probas = torch.softmax(x, dim=1)
        return x

In [ ]:
model = AllConvNet(num_classes=10)
model = model.to(DEVICE)

optimizer = torch.optim.SGD(model.parameters(), momentum=0.9, lr=0.1)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                       factor=0.1,
                                                       mode='max',
                                                       verbose=True)

minibatch_loss_list, train_acc_list, valid_acc_list = train_model(
    model=model,
    num_epochs=NUM_EPOCHS,
    train_loader=train_loader,
    valid_loader=valid_loader,
    test_loader=test_loader,
    optimizer=optimizer,
    device=DEVICE,
    scheduler=scheduler,
    scheduler_on='valid_acc',
    logging_interval=100)

plot_training_loss(minibatch_loss_list=minibatch_loss_list,
                   num_epochs=NUM_EPOCHS,
                   iter_per_epoch=len(train_loader),
                   results_dir=None,
                   averaging_iterations=200)
plt.show()

plot_accuracy(train_acc_list=train_acc_list,
              valid_acc_list=valid_acc_list,
              results_dir=None)
plt.ylim([60, 100])
plt.show()

In [ ]:
model.cpu()
unnormalizer = UnNormalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
class_dict = {0: 'airplane',
              1: 'automobile',
              2: 'bird',
              3: 'cat',
              4: 'deer',
              5: 'dog',
              6: 'frog',
              7: 'horse',
              8: 'ship',
              9: 'truck'}


In [ ]:
mat = compute_confusion_matrix(model=model, data_loader=test_loader, device=torch.device('cpu'))
plot_confusion_matrix(mat, class_names=class_dict.values())
plt.show()